# Shot overview

In [ ]:
%matplotlib notebook
from IPython.core.display import display, HTML
import ipfnpytools.aug_read as aug_read
from ipfnpytools.plot import plot_signals
from ipfnpytools.save_figure import save_figure
import numpy as np
import pandas as pd
from collections import namedtuple
import matplotlib.pyplot as plt

def valid(x):
    return True if x is not None and np.isfinite(x) else False

## Parameters

Change the parameters in the next cell to suit your needs

### Choosing the signals

To add signals you must create an new row following the suggested structure. You must fill-in every parameter:
* **diagnostic:** three-letter shotfile identifier
* **name:** signal identifier within the shotfile
* **scale:** scale factor, e.g., 1e6
* **y_min / y_max:** lower and upper limits for the plot. Choose `None` to enable auto range.
* **box:** specify in which box to draw the signal
* **y_label:** label to put on the yy axis. Supports latex syntax, e.g., D-\\$\alpha\\$ => D-$\alpha$ 
* **line_width:** line width for the signal. Should be small (<1) for signals with fast fluctuations.

To remove signals, simply comment the corresponding line with a `#`. To comment/uncomment multiple lines simultaneously, select the lines and press `Ctrl`+`~`

In [ ]:
shot = 37447

t_begins = 0 # Begin at time t_begins
t_ends = 8 # End at time t_ends

figure_size = (9, 11)  # Figure size in inches (width, height)

show_columns = 1  # Number of columns in which to show the data

signals = pd.DataFrame(
    
columns=
['diagnostic', 'name', 'scale', 'y_min', 'y_max', 'box', 'y_label', 'line_width'],
    
data=[                          
['MAG', 'Ipa'     ,  1e6, None, None, 0,                             "$MA$", 1.5],  # Current
['NIS', 'PNI'     ,  1e6, None, None, 1,                             "$MW$", 1.5],  # Neutral beam power
['ECS', 'PECRH'   ,  1e6, None, None, 1,                             "$MW$", 1.5],  # ECRH power
['DCN', 'H-1'     , 1e19, None, None, 2, "$\overline{n_e}$ ($10^{19}/m^{3})$", 1],    # Horizontal interferometer
['DCN', 'H-5'     , 1e19, None, None, 2, "$\overline{n_e}$ ($10^{19}/m^{3})$", 1],    # 45 degree upper interferometer
# ['FPG', 'delRoben',    1, None, None , 9,                                   "Triang.", 1],  # Triangularity
# ['FPG', 'Zgeo'    ,    1, None, None, 3,                                   "z [m].", 1],  # Triangularity
# ['FPG', 'k'       ,    1, None, None, 4,                                   "Elong.", 1],  # Elongation
# ['FPG', 'q95'     ,    1, None, None, 5,                                   "q95", 1],  # Elongation
# ['TOT', 'Wmhd'    ,  1e6, None, None, 2,                     "$W_{mhd}$ (MJ)", 1],
# ['FPC', 'IpiFP'   ,  1e6, None, None, 3,                       "$I_{p}$ (MA)", 1],
['POT', 'ELMa-Han',    1, None, None, 3,                 r"$D_\alpha$ (a.u.)", 0.1],
['POT', 'ELMi-Han',    1, None, None, 3,                 r"$D_\alpha$ (a.u.)", 0.1],
# ['TOT', 'P_OH'    ,  1e6, None, None, 5,                             "P (MW)", 1],
# ['ECS', 'PECRH'   ,  1e6, None, None, 5,                             "P (MW)", 1],
# ['BPD', 'Pradtot' ,  1e6, None, None, 5,                             "P (MW)", 1],
['MAC', 'Ipolsola',  1e3,  None, None, 4,                     "$I_{div}$ (kA)", 0.1],  # Outer div. current
['MAC', 'Ipolsoli',  1e3,  None, None, 4,                     "$I_{div}$ (kA)", 0.1],  # Inner div. current
# ['UVS', 'D_tot'   , 1e21, None, None, 7,                 "Fuel ($10^{21}$/s)", 2],  # Deuterium injection
# ['UVS', 'N_tot'   , 1e21, None, None, 4,                 "Fuel ($10^{21}$/s)", 2],  # N injection
# ['UVS', 'CFDu13B' , 1e21, None, None, 2,                 "Fuel ($10^{21}$/s)", 2],
# ['UVS', 'CFA13A'  , 1e21, None, None, 2,                 "Fuel ($10^{21}$/s)", 2],
['UVS', 'H_tot'   , 1e21, None, None, 5,                 "Fuel ($10^{21}$/s)", 1],  # Hydrogen injection
# ['DDS', 'Tdiv'    ,    1, None, None, 8,                              "eV???", 1]
])

## Confirm the signal selection

In [ ]:
# Confirm everything is has you want
signals

## Load the signals from AFS

In [ ]:
data = aug_read.many_signals(
    diagnostics=signals['diagnostic'].tolist(),
    names=signals['name'].tolist(),
    shots=shot)

## Plot signals

In [ ]:
fig, axes, fig_name = plot_signals(
    times = data.times, signals = data.signals,
    diagnostics = signals['diagnostic'].tolist(),
    names=signals['name'].tolist(), 
    boxes=signals['box'].tolist(), 
    signal_multipliers=[1.0/x for x in signals['scale'].tolist()],
    y_labels=signals.sort_values(by='box')['y_label'].unique(),
    postfixes="overview", ncols=show_columns,
    order='row',
    t_begins=t_begins, t_ends=t_ends,
    hspace = 0.1,
    title=str(shot),
    linewidths=signals['line_width'].tolist(),
)

fig.set_size_inches(*figure_size)

# Set proper limits and labels 
for index, row in signals.iterrows():
    axes.flatten()[row['box']].set_ylim(row['y_min'] if valid(row['y_min']) else None,
                              row['y_max'] if valid(row['y_max']) else None)

**Run the cell bellow to save the figure.** File name is automatic. To customize the file name, replace `fig_name` with an apropriate name, e.g, `"my_figure"`

In [ ]:
save_figure(fig, fig_name)